In [ ]:
# 資料庫串接
import MySQLdb
import pandas as pd
conn = MySQLdb.Connect(host = 'localhost',
                    port = 3306,
                    user = 'root',
                    password = 'Terry970135',
                    db = 'kpmg',
                    charset='utf8')
# 輸入你的資料庫連線資訊
cur = conn.cursor()
company_id_input = 2707
# kind_input = "Balance_Sheet" #你要查的表
# kind_input = "Profit_and_Loss_Account" #你要查的表
# kind_input = "Cash_Flow_Statement" #你要查的表
kind_input = "Statement_of_changes_in_equity" #你要查的表


year_input = 2019
season_input = 4
cur.execute(f"SELECT `path` FROM `financial_statements` WHERE `company_id` = {company_id_input} AND `kind` = '{kind_input}' AND `year` = {year_input} AND `season` = {season_input} ")
result_set = cur.fetchall()
# result_set
df = pd.read_csv(f"../../{result_set[0][0]}")

In [ ]:
其他權益項目合計 = {"其他權益項目合計":"透過其他綜合損益按公允價值衡量之金融資產未實現評價(損)益"} 

# 橫的
普通股股本 = {"普通股股本":"股本合計"}

保留盈餘合計 = {"保留盈餘合計":"法定盈餘公積+特別盈餘公積+未分配盈餘（或待彌補虧損）"}

歸屬於母公司業主之權益總計={"歸屬於母公司業主之權益總計":"股本合計+資本公積+保留盈餘合計+其他權益項目合計"}

權益總額 = {"權益總額":"歸屬於母公司業主之權益總計+非控制權益"}
# 橫的

本期綜合損益總額_1 = {"本期綜合損益總額 ":"本期淨利+本期其他綜合損益"}

期末餘額 = {"期末餘額 ":"期初餘額+權益增加"}

權益增加_減少_總額 = {"權益增加(減少)總額":"提列特別盈餘公積+普通股現金股利+本期淨利+本期其他綜合損益+非控制權益增減"}

In [ ]:
import pandas as pd
import os
import re
import numpy as np
def clean_df(df):
    df = df.dropna(axis=1)
    df_cl_ls = list(df.columns)
    for i in range(3,len(df_cl_ls)):
        s = re.sub('[0-9A-Za-z]',"",df_cl_ls[i])
        s = s.replace(" ","")
        s = s.replace("(","")
        s = s.replace(")","")
        s = s.replace("-","")
        df_cl_ls[i]=s
    df.columns = df_cl_ls  
    return df

def vertival_search_result(search_target):
    filt = (new_df["會計項目"].str.contains(search_target))
    x = new_df.loc[filt]
    num_ls = np.array(x.iloc[0])[3:]
    for i in range(len(num_ls)):
        if type(num_ls[i]) != int:
            try:
                if "(" in num_ls[i]:
                    num_ls[i] = num_ls[i].replace("(","")
                    num_ls[i] = num_ls[i].replace(")","")
                    num_ls[i] = num_ls[i].replace(",","")
                    num_ls[i] = -(int(num_ls[i]))
            except:
                pass
            num_ls[i]=int(num_ls[i])
    return num_ls

def calculate(new_df):
    new_dict = {}
    kind_list = []
    money_list = []
    status_list = []
    horizontal_function = [保留盈餘合計,歸屬於母公司業主之權益總計,權益總額,普通股股本]
    # vertical_function = [普通股股本]
    vertical_function = [本期綜合損益總額_1,期末餘額,權益增加_減少_總額]
    # horizontal_function = [權益總額]
    h_ls = ["期初餘額","期末餘額"]
    new_df_cl = list(new_df.columns)
    for i in horizontal_function:
        for h_col in h_ls:
            temp_ls = []
            filt = (new_df["會計項目"].str.contains(h_col))
            x = new_df.loc[filt]
            for k, v in i.items():
                v_ls = re.split(r"-|\+",v)
                for kind in v_ls:
                    try:
                        num = new_df_cl.index(kind)
                        temp_num = x.iloc[0,[num]][0]
                        if "(" in str(temp_num):
                            temp_num = temp_num.replace("(","")
                            temp_num = temp_num.replace(")","")
                            temp_num = temp_num.replace(",","")
                            temp_num = -(int(temp_num))
                        temp_num = int(temp_num)
                        temp_ls.append(temp_num)
                    except:
                        pass
            temp_sum = sum(temp_ls)
            target = new_df_cl.index(k)
            target_sum = x.iloc[0,[target]][0]
            if "(" in str(target_sum):
                target_sum = target_sum.replace("(","")
                target_sum = target_sum.replace(")","")
                target_sum = target_sum.replace(",","")
                target_sum = -(int(target_sum))
            else:
                target_sum = int(target_sum)
            kind_list.append(h_col+"_"+k)
            money_list.append(target_sum)
            if temp_sum == target_sum:
                status_list.append(True)
            else:
                status_list.append(False)
    for v_d in vertical_function:
        vertical_temp_sum = []
        for k, v in v_d.items():
            f_ls = v.split("+")
            for each_k in f_ls:
                try:
                    result_num_ls = vertival_search_result(each_k)
                    vertical_temp_sum.append(result_num_ls)
                except:
                    pass
        vertical_result_sum = sum(vertical_temp_sum)
        print(vertical_result_sum)
        try:
            target_result_sum = vertival_search_result(k)
            if target_result_sum.all() == vertical_result_sum.all():
                status_list.append(True)
            else:
                status_list.append(False)
            kind_list.append(k)
            money_list.append(target_result_sum)
        except:
            pass

    new_dict["會科"]=kind_list
    new_dict["金額"]=money_list
    new_dict["符合"]=status_list
    result_df = pd.DataFrame(new_dict)
    return result_df

# company_id_input = [2707,2727,2739]
# season_input = 4
# for each_co_id in company_id_input:
#     try:
#         os.mkdir(f"statement_result/{each_co_id}")
#     except:
#         pass
#     for year_input in range(2019,2022):
#         df = connect_db(each_co_id,year_input,4)
new_df = clean_df(df)
result = calculate(new_df)
result.to_csv(f"zxc.csv",encoding="utf-8-sig")

In [ ]:
filt = (new_df["會計項目"].str.contains("本期綜合損益總額"))
x = new_df.loc[filt]
num_ls = np.array(x.iloc[0])[3:]
for i in range(len(num_ls)):
    if type(num_ls[i]) != int:
        try:
            if "(" in num_ls[i]:
                num_ls[i] = num_ls[i].replace("(","")
                num_ls[i] = num_ls[i].replace(")","")
                num_ls[i] = num_ls[i].replace(",","")
                num_ls[i] = -(int(num_ls[i]))
        except:
            pass
        num_ls[i]=int(num_ls[i])
num_ls

In [ ]:
c = vertival_search_result("本期綜合損益總額")
c

In [ ]:
import numpy as np
s = np.array([1,2])+np.array([3,4])

In [ ]:
new_df_cl = list(new_df.columns)
new_df_cl

In [ ]:
i = "期初餘額"
filt = (new_df["會計項目"].str.contains(i))
x = new_df.loc[filt]
kind = "非控制權益"
num = new_df_cl.index(kind)
x.iloc[0,[num]][0]
# x.iloc[0,[3]][0]

In [ ]:
v = 保留盈餘合計["保留盈餘合計"]
v = re.split(r"-|\+",v)
for i in v:
    print(i)
    print(new_df_cl.index(i))

In [ ]:
import pandas as pd

v = x
v = v.dropna(axis=1)
c = v.values
c[0][3:]

In [ ]:
import re
h = list(x.columns)
for i in range(3,len(h)):
    s = re.sub('[0-9A-Za-z]',"",h[i])
    s = s.replace(" ","")
    h[i]=s

In [ ]:
x.columns = h

In [ ]:
x

In [ ]:
# 3310法定盈餘公積 +3320特別盈餘公積 +3350未分配盈餘（或待彌補虧損） =3300保留盈餘合計 
# 31XX歸屬於母公司業主之權益總計=3100股本合計 +3200資本公積 +3300保留盈餘合計 +3400其他權益項目合計 

# 31XX歸屬於母公司業主之權益總計 +36XX非控制權益  =3XXX權益總額



In [ ]:
# D1本期淨利（淨損 )+D3本期其他綜合損益 =D5本期綜合損益總額 

# A1期初餘額+Y1權益增加(減少)總額 =Z1期末餘額 

# B3提列特別盈餘公積 +B5普通股現金股利 +D1本期淨利（淨損 )+D3本期其他綜合損益+O1非控制權益增減 =Y1權益增加(減少)總額

In [ ]:
new_df

In [ ]:
import numpy as np

a = [np.array([1,2])+np.array([3,4])]
s = sum(a)
if s.all() == s.all():
    print(123)